In [ ]:
# # Iterate over each chunk
# for start in range(1, total_files + 1, chunk_size):
#     end = min(start_id + chunk_size - 1, total_files)

In [74]:
import os
import numpy as np
import nibabel as nib
import shutil
import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard



def load_nifti_file(filepath):
    """Load a NIfTI file and return its data as a numpy array."""
    scan = nib.load(filepath)
    return scan.get_fdata()

def normalize(volume):
    """Normalize the volume by clipping and standardizing."""
    min_val = -1000
    max_val = 400
    volume = np.clip(volume, min_val, max_val)
    mean = np.mean(volume)
    std = np.std(volume)
    volume = (volume - mean) / std
    return volume

input_dir = './Dataset'
output_dir = './Dataset/Processed'
total_size={'Training':369,'Validation':125}
raw_path = {
    'Training': {
        'flair': 'flair',
        't1': 't1',
        't1ce': 't1ce',
        't2': 't2',
        'mask': 'seg'
    },
    'Validation': {
        'flair': 'flair',
        't1': 't1',
        't1ce': 't1ce',
        't2': 't2'
    }
}
def preprocess_train_chunk(input_dir, output_dir, phase,chunk_index):
    
        chunk_size=41
        X_train = np.empty(chunk_size, 240, 240, 155,4)
        y_train = np.empty(chunk_size,240, 240, 155,1, dtype=int)
        start_id = chunk_index * chunk_size + 1
        end_id = min((chunk_index + 1) * chunk_size, 369)
        for i in range(start_id, end_id + 1):
            h=0
            id = f"{i:03d}"
            npy_folder = os.path.join(output_dir, phase, f'chunk_{chunk_index}',i)
            os.makedirs(npy_folder, exist_ok=True)
            for j,modality in enumerate( ['flair', 't1', 't1ce', 't2','seg']):
                if modality != 'seg':
                    #Modality data
                    channel_name = f'BraTS20_{phase}_{id}_{modality}'
                    channel_path = os.path.join(input_dir, phase, modality)
                    channel = load_nifti_file(os.path.join(channel_path, channel_name+ '.nii'))
                    channel = normalize(channel)
                    npy_channel_path=os.path.join()
                    npy_channel=np.save(npy_channel_path, channel)
                    X_train[h, :, :, :, j] = np.load(npy_channel)   
                else:
                    #Mask data
                    mask_path= os.path.join(input_dir, phase, 'mask')
                    mask_name= f'BraTS20_{phase}_{id}_{seg}'
                    mask = load_nifti_file(os.path.join(mask_path, mask_name + '.nii'))
                    npy_mask_path=os.path.join()
                    npy_mask=np.save(npy_mask_path, mask)
                    npy_mask=np.load(npy_mask)
                    y_train[h,] = npy_mask.reshape(*self.dim, 1)  
                h=+1




            
            